In [2]:
using SymPy
using LinearAlgebra
using Plots
using Combinatorics
using TimerOutputs
using LowRankApprox

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1317


In [2]:
 x = Sym("x")
y = Sym("y")
r = Sym("r")
r2 = Sym("r2")

r₂

In [3]:
function plot_poly(poly, a, b, show_true = false)
    f = lambdify(poly, [r])
    delta=(b-a)/1000
    x_vals = collect(a:delta:b)
    y_vals = map(f, x_vals)
    
    g = lambdify(1/(1+x^2), [x])
    true_vals = map(g, x_vals)
    if show_true
        return plot(x_vals, hcat(y_vals, true_vals),  display=true)
    else
        return plot(x_vals, y_vals,  display=true)
    end
end



plot_poly (generic function with 2 methods)

In [4]:
p_size = 500
pij = zeros(Float64, p_size,p_size)
pij[1,1] = 1
pij[2,2] = 1
for i in 3:p_size
    for j in 1:i
        if j == 1
            pij[i,j] = -pij[i-2,j]
        else
            pij[i,j] = 2pij[i-1,j-1] - pij[i-2,j]
        end
    end
end

        
            

In [5]:
function cheb_polys(degree, b)
    polys = []
    for n in 0:degree
        poly = 0
        for k in 0:n
            poly += pij[n+1,k+1]*(r/b)^k
        end
        push!(polys, expand(poly))
    end
    return polys
end

cheb_polys (generic function with 1 method)

In [6]:
function dct(kern, k, b, dct_n)
    total = 0
#     js = collect(0:N)
#     arg1 = kern.(b*cos.(pi*js/N))
#     arg2 = cos.(pi*k*js/N)
#     total = dot(arg1,arg2) - 0.5(arg1[1]*arg2[1]+arg1[end]*arg2[end])
    for j in 0:dct_n
        val = kern(b*cos(pi*j/dct_n)) * cos(pi*k*j/dct_n)
        if j==0 || j==dct_n
                val*=0.5
        end
        total += val
    end
    return (2.0/dct_n)*total
end

dct_n         = 100 # Iterations for discrete cosine transform
d             = 3
kern          = (1+abs(r))*exp(-abs(r))
mat_kern(x,y) = (1+norm(x-y))*exp(-norm(x-y))
lkern         = lambdify(kern)


#101 (generic function with 1 method)

In [7]:
function delta(x,y)
    if x==y return 1 else return 0 end
end

delta (generic function with 1 method)

In [8]:

function transform_coef(k3, k2, k1, degree, pij, a_vals, b, xp)
    tot3 = 0
    for i in convert(Int,2k1+2k2+2k3):degree
        tot3 +=(pij[i+1, convert(Int, 2k1+2k2+2k3+1)]
            *a_vals[i+1]
            *(1-delta(0,i)/2))
    end
    return tot3 * multinomial(xp...)*multinomial([k1,k2,k3]...)*((1.0/b)^(2k1+2k2+2k3))*((-2.0)^k3)
end

transform_coef (generic function with 1 method)

In [9]:
# function get_pole_map(degree, d)
#     many_to_idx = Dict()
#     count=0
#     for k3 in 0:(degree/2)
#         for xp in multiexponents(d,convert(Int,k3))   
#             for k2 in 0:convert(Int, (floor(degree/2)-k3)) 
#                 count+=1
#                 many_to_idx[k3, xp, k2]=count
#             end
#         end
#     end
#     return many_to_idx
# end

In [10]:
struct fkt_config
    fkt_deg::Int
    d::Int
    b::Float64
    dct_n::Int
    to::TimerOutput
end

In [45]:
function cauchy_fkt(lkern, x_vecs, y_vecs, truth_mat, fkt_config)
    degree = fkt_config.fkt_deg
    d      = fkt_config.d
    b      = fkt_config.b
    dct_n  = fkt_config.dct_n
    to     = fkt_config.to
    
    num_points = length(x_vecs)
    pole_count = binomial(convert(Int, floor(degree/2))+d+1,d+1)
    a_vals = zeros(degree+1) # kern's coefs in cheb poly basis
    for i in 0:(degree)
        a_vals[i+1] = dct(lkern, i, b, dct_n)
    end
    
    U_mat = zeros(num_points,pole_count)
    U_mat_parts = zeros(degree+1, num_points,pole_count)
    V_mat = zeros(pole_count,num_points)
    pole_counter=0

    @timeit to string("fkt, n=", num_points, ", deg=", degree) begin
        for k3 in 0:convert(Int,(degree/2))
            for xp in multiexponents(d,convert(Int,k3))                
                for k2 in 0:convert(Int, (floor(degree/2)-k3))
                    pole_counter +=1
                    @timeit to "x loop" begin
                    for x_idx in 1:length(x_vecs)
                        x_sum = 0  
                        for i in convert(Int,2k2+2k3):degree
                            tmp = 0
                            for k1 in 0:convert(Int, floor(i/2)-k2-k3)
#                             tot3 = 0
                                tmp +=((norm(x_vecs[x_idx])^(2k1))
                                        *prod(x_vecs[x_idx].^(xp))
                                        *multinomial(xp...)
                                        *pij[i+1, convert(Int, 2k1+2k2+2k3+1)]
                                        *(1-delta(0,i)/2)
                                        *multinomial([k1,k2,k3]...)
                                        *((1.0/b)^(2k1+2k2+2k3))
                                        *((-2.0)^k3))
                            end
                            U_mat_parts[i+1, x_idx, pole_counter] = tmp
                            U_mat[x_idx, pole_counter] += a_vals[i+1]*tmp
#                             tc = tot3 
#                             @timeit to "tc" tc=transform_coef(k3, k2, k1, degree, pij, a_vals, b, xp)
#                             x_sum +=(norm(x_vecs[x_idx])^(2k1))*tc
                        end
#                         U_mat[x_idx, pole_counter] = x_sum*prod(x_vecs[x_idx].^(xp))
                    end
                        end
                    @timeit to "y loop" begin
                        for y_idx in 1:length(y_vecs)
                            V_mat[pole_counter, y_idx] = ((norm(y_vecs[y_idx])^(2k2))*prod(y_vecs[y_idx].^(xp)))
                        end
                    end
                end
            end
        end
    end
    guess = U_mat*V_mat
    error = norm(guess-truth_mat)/norm(truth_mat)    
    
    idf = idfact(transpose(V_mat), rtol = 1e-6)
#     approx_mat[:, idf.sk] .= truth_mat[:, idf.sk]
    println(size(V_mat,1), " ", size(V_mat, 2))
    println(length(idf.sk))
    println(idf.sk)
    for i in 1:length(idf.sk)
        println(norm(V_mat[idf.sk[i], :]))
    end
    
    
#     println("\n\nSTATS")
#     println("guess mat rank ", rank(guess, rtol=1e-6))
#     println("u_mat width ", size(U_mat, 2))
#     println("u mat rank ", rank(U_mat, rtol=1e-6))
#     println("v mat rank ", rank(V_mat, rtol=1e-6))
#     for i in 0:degree
#         println(i)
#         println("u mat part rank ", rank(U_mat_parts[i+1, :, :], rtol=1e-6))
#         println("uv mat part rank ", rank(U_mat_parts[i+1, :, :]*V_mat, rtol=1e-6))
        
#     end
    #     push!(errors, norm(guess-truth_mat)/norm(truth_mat))
# #     guess_arr = []
# #     for (x_idx, x_vec) in enumerate(x_vecs)
# #         for (y_idx, y_vec) in enumerate(y_vecs)
# #             push!(guess_arr, guess[x_idx, y_idx])
# #         end
# #     end
# #     push!(guesses, guess_arr)
# #     println("Guess mat actual rank is ", rank(guess, rtol=1e-6))
# #     println("U is ", size(U_mat), " rank is ", rank(U_mat, rtol=1e-6))
# #     println("V is ", size(V_mat), " rank is ", rank(V_mat, rtol=1e-6))
# #     push!(U_mats, U_mat)
# #     push!(V_mats, V_mat)
# #     test_guess = U_mat*(V_mat*test_vec)
# #     push!(errors, norm(test_guess-test_truth)/norm(test_truth))

# #     for i in 1:size(truth_mat, 1)
# #         guess[i,i] = truth_mat[i,i]
# #     end
# #     test_guess =guess*test_vec
# #     push!(adj_errors,  norm(test_guess-test_truth)/norm(test_truth))
    return U_mat, V_mat, error
end

cauchy_fkt (generic function with 1 method)

# Varying N

In [32]:
for y in  multiexponents(3,0)
#     if y == zeros(3)
        println(y)
#     end
end


[0, 0, 0]


In [ ]:
to      = TimerOutput()
idto    = TimerOutput()
iderrs  = []
fkterrs = []
ranks   = []

fkt_deg = 6
point_range = collect(1000:2000:10000)
for num_points in point_range
    println(num_points)
    x_vecs = [randn(d)./8 for _ in 1:num_points]
    y_vecs = [randn(d)./8 for _ in 1:num_points]
    max_norm = max(maximum(norm.(x_vecs)), maximum(norm.(y_vecs)))
    truth_mat  = mat_kern.(x_vecs, permutedims(y_vecs))
    cfg = fkt_config(fkt_deg, d, 2max_norm, dct_n, to)
    umat, vmat, err = cauchy_fkt(lkern, x_vecs, y_vecs,truth_mat, cfg)
    push!(fkterrs, err)
    
    r = size(umat, 2)
    push!(ranks, r)
    @timeit idto string("n=",num_points) idf = idfact(truth_mat, rank = r)
    approx_mat = deepcopy(truth_mat)
#     approx_mat[:, idf.sk] .= truth_mat[:, idf.sk]
    approx_mat[:, idf.rd] .= truth_mat[:, idf.sk]*idf.T
    push!(iderrs, (norm(approx_mat - truth_mat))/norm(truth_mat))
end

# test_vec   = randn(num_points)
# test_truth = truth_mat*test_vec
# truth_vec = []
# r_vec     = []
# for (x_idx, x_vec) in enumerate(x_vecs)
#     for (y_idx, y_vec) in enumerate(y_vecs)
#         push!(truth_vec, truth_mat[x_idx, y_idx])
#         push!(r_vec, norm(x_vec-y_vec))
#     end
# end
# errors     = []
# adj_errors = []
# ranks      = []
# guesses    = []
# U_mats = []
# V_mats = []

In [ ]:
println(fkterrs)

In [ ]:
# plot(point_range, fkterrs, yaxis=:log, label="fkt")
# # plot!(ranks, adj_errors, yaxis=:log, label="adjfkt")
# plot!(point_range, iderrs, yaxis=:log, label="id", xlabel="Rank", ylabel="Relative error")


In [ ]:
fkt_times = [TimerOutputs.time(to[string("fkt, n=", range, ", deg=", fkt_deg)])/(10^9) for range in point_range]
id_times = [TimerOutputs.time(idto[string("n=", range)])/(10^9) for range in point_range]
plot(point_range, fkt_times,  label="fkt")
# plot!(ranks, adj_errors, yaxis=:log, label="adjfkt")
plot!(point_range, id_times,  label="id", xlabel="Num points", ylabel="Time (s)")


# Varying degree

In [46]:
to      = TimerOutput()
idto    = TimerOutput()
iderrs  = []
fkterrs = []
ranks   = []

# fkt_deg = 6
num_points = 2000
# x_vecs = [randn(d)./8 for _ in 1:num_points]
# y_vecs = [randn(d)./8 for _ in 1:num_points]
# max_norm = max(maximum(norm.(x_vecs)), maximum(norm.(y_vecs)))
# truth_mat  = mat_kern.(x_vecs, permutedims(y_vecs))

x_vecs = [randn(d)./8 for _ in 1:num_points]
# y_vecs = [randn(d)./8 for _ in 1:num_points]
max_norm = max(maximum(norm.(x_vecs)), maximum(norm.(x_vecs)))
truth_mat  = mat_kern.(x_vecs, permutedims(x_vecs))

# for fkt_deg in 2:2:14
umat = []
vmat = []
for fkt_deg in [14]
    println(fkt_deg)
    cfg = fkt_config(fkt_deg, d, 2max_norm, dct_n, to)
    umat, vmat, err = cauchy_fkt(lkern, x_vecs, x_vecs, truth_mat, cfg)
    push!(fkterrs, err)
    
    r = size(umat, 2)
    push!(ranks, r)
    @timeit idto string("n=",num_points, ", deg=", fkt_deg) idf = idfact(truth_mat, rank = r)
    approx_mat = deepcopy(truth_mat)
#     approx_mat[:, idf.sk] .= truth_mat[:, idf.sk]
    approx_mat[:, idf.rd] .= truth_mat[:, idf.sk]*idf.T
    push!(iderrs, (norm(approx_mat - truth_mat))/norm(truth_mat))
end

14
330 2000
105
[1, 23, 16, 9, 2, 60, 42, 54, 36, 30, 24, 10, 17, 66, 111, 96, 3, 76, 71, 91, 86, 172, 43, 116, 156, 55, 37, 124, 160, 25, 49, 140, 11, 168, 61, 18, 120, 144, 236, 176, 221, 4, 182, 233, 179, 82, 87, 102, 218, 224, 185, 194, 293, 56, 188, 117, 44, 212, 72, 281, 197, 215, 38, 50, 279, 26, 283, 12, 243, 239, 19, 241, 291, 269, 289, 259, 129, 149, 145, 330, 247, 271, 323, 267, 5, 265, 295, 273, 133, 275, 255, 322, 108, 103, 68, 88, 297, 296, 329, 207, 324, 113, 325, 83, 33]
44.721359549995796
5.654108281667178
5.637353725549871
5.606293954651411
2.7612462231351325
1.2742617144340749
0.7348837851427766
0.7206809883893169
0.6937640930450275
1.2248685380992557
0.562128576163077
0.5342965996764613
0.5053884402406488
0.34232305797008444
0.3711675689911134
0.3154230395872095
0.34884961299478306
0.1673804887022388
0.15224872773608347
0.16758429945600067
0.09282951901578396
0.12561439725389276
0.09892026102643996
0.11035056704607492
0.09571964377653952
0.09276081592708722
0.084701

In [26]:
# a_vals = zeros(12+1) # kern's coefs in cheb poly basis
# for i in 0:(12)
#     a_vals[i+1] = dct(lkern, i, 2max_norm, dct_n)
# end
# a_vals
test_vec = randn(size(truth_mat,1))
guess_vec = umat*vmat*test_vec
truth_vec = truth_mat*test_vec
println("Error ", norm(guess_vec-truth_vec)/norm(truth_vec))
# gsvd = svdvals(guess)
# tsvd = svdvals(truth_mat)
# plot(1:length(gsvd), gsvd, yscale = :log10)
# plot!(1:length(tsvd), tsvd, yscale = :log10)


Error 5.216700549952521e-5


In [27]:
guess = umat*vmat
for i in 1:size(guess,1)
    guess[i,i] = truth_mat[i,i]
end
test_vec = randn(size(truth_mat,1))
guess_vec = guess*test_vec
truth_vec = truth_mat*test_vec
println("Error ", norm(guess_vec-truth_vec)/norm(truth_vec))


Error 8.400717486373816e-5


In [16]:
guesssvd

2000-element Vector{Float64}:
 1926.4666135037698
   22.304912127827567
   21.806983635256827
   21.30256136215557
    1.6925595976972194
    0.868895207924722
    0.8402025395459152
    0.8346407474163089
    0.8153894574599159
    0.7909322032425686
    0.2538441911784699
    0.23356852985033616
    0.2263707422877834
    ⋮
    1.7247666901504095e-16
    1.574455075500982e-16
    1.4385508540647508e-16
    1.3795436288167145e-16
    1.2657000495088116e-16
    9.901882101908024e-17
    8.145293378519483e-17
    7.202321649464876e-17
    5.426500632797803e-17
    3.1547212538765665e-17
    1.204382336404685e-17
    5.968493253676047e-18

In [ ]:
fkt_times = [TimerOutputs.time(to[string("fkt, n=5000, deg=",fkt_deg)])/(10^9) for fkt_deg in 2:2:14]
id_times = [TimerOutputs.time(idto[string("n=5000, deg=", fkt_deg)])/(10^9) for fkt_deg in 2:2:14]
plot(ranks, fkt_times,  label="fkt")
# plot!(ranks, adj_errors, yaxis=:log, label="adjfkt")
plot!(ranks, id_times,  label="id", xlabel="Rank", ylabel="Time (s)")


In [ ]:
plot(ranks, fkterrs, yscale=:log10, label="fkt")
plot!(ranks, iderrs, yscale=:log10, label="id", xlabel="Rank", ylabel="Relative error (frob)")


In [ ]:
# So most rs are in 0-0.75, yet our polys are -1 to 1 ish. 
# Error in exp peaks around same as histogram >:|
histogram(r_vec, xlabel="r")

In [ ]:
plt = nothing
stride = 1000
for k in 5:length(guesses)
    errs = [guess[1:stride:length(guess)] - truth_vec[1:stride:length(guess)] for guess in [guesses[k]]]
    gs = [guess[1:stride:length(guess)] for guess in [guesses[k]]]
    truths = [truth_vec[1:stride:length(guess)] for guess in [guesses[k]]]
    if k==5
        plt=scatter([r_vec[1:stride:length(r_vec)]],errs, label=k)
#         plt=scatter([r_vec[1:stride:length(r_vec)]],gs, label=k)
#         plt=scatter!([r_vec[1:stride:length(r_vec)]],truths, label=k)
    else
        plt=scatter!([r_vec[1:stride:length(r_vec)]],errs, label=k)

#         plt=scatter!([r_vec[1:stride:length(r_vec)]],gs, label=k)
#         plt=scatter!([r_vec[1:stride:length(r_vec)]],truths, label=k)
    end
end
plt
# plt=plot!([r_vec[1:stride:length(r_vec)]],zeros(length(1:stride:length(r_vec))), label=0)


In [ ]:
# try diagonal or sparse correction
# issue - the matrix is low rank, diagonal correction doesn't actually help much 

# QR with polys in chebfun, get error, truncate

In [ ]:
plot(ranks, errors, yaxis=:log, label="fkt")
plot!(ranks, svd_errs, yaxis=:log, label="svd")
plot!(idranks, iderrs, yaxis=:log, label="id")


In [ ]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################
# function gen_binom(n, k)
#     numer = prod([Rational{BigInt}(n)-i for i in 0:(k-1)])
#     denom = factorial(big(k))
#     return numer//denom
# end
# function init_P_mat(degree)
#     P = Matrix(undef, degree+1, degree+1)
#     for i in 0:degree
#         for j in 0:degree
#             P[1+i, 1+j] = 2^i*binomial(i,j)*gen_binom((i+j-1)//2,i)
#         end
#     end
#     return P
# end
# function init_B_mat(a,b, degree)
#     B = Matrix(undef, degree+1, degree+1)
#     for i in 0:degree
#         for j in 0:degree
#             if abs(a)==abs(b) && j!=i
#                 B[1+i, 1+j] =0
#             else
#                 B[1+i, 1+j] = (2/(b-a))^j*(-((2a/(b-a))+1))^(i-j)*binomial(i,j)
#             end
#         end
#     end
#     return B
# end

